In [19]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('processed_data.csv')

In [7]:
df.columns

Index(['Дата', 'ДатаДоставки', 'НомерЗаказаНаСайте', 'НовыйСтатус',
       'СуммаЗаказаНаСайте', 'СуммаДокумента', 'МетодДоставки', 'ФормаОплаты',
       'Регион', 'Группа2', 'Группа3', 'Группа4', 'Тип', 'Номенклатура',
       'ТипТовара', 'Отменено', 'ПричинаОтмены', 'Количество', 'Цена',
       'СуммаСтроки', 'ЦенаЗакупки', 'МесяцДатыЗаказа', 'ГодДатыЗаказа',
       'ПВЗ_код', 'Статус', 'Гео', 'СуммаУслуг', 'СуммаДоставки',
       'КоличествоПроданоКлиенту', 'ДатаЗаказаНаСайте', 'Телефон_new',
       'ЭлектроннаяПочта_new', 'Клиент', 'ГородМагазина', 'Полученная маржа',
       'Размещенная маржа', 'Товар выкуплен', 'Товар выкуплен частично',
       'Товар не выкуплен', 'Товар размещен', 'Остаток', 'Пол клиента',
       'Размещенная выручка', 'Полученная выручка'],
      dtype='object')

In [9]:
dataset = df[['Телефон_new', 'Группа4']]

In [12]:
ds_clients, clients_names = pd.factorize(dataset['Телефон_new'])

In [13]:
len(clients_names)

221921

In [14]:
len(ds_clients)

1346913

In [15]:
ds_goods, goods_names = pd.factorize(dataset['Группа4'])

In [17]:
len(goods_names) * len(clients_names) * 4 / 1024 / 1024

380.10608291625977

In [21]:
user_items = np.zeros(shape=(len(clients_names), len(goods_names)))

In [24]:
user_items[ds_clients, ds_goods] = 1

In [27]:
user_items.tocsr()

AttributeError: 'numpy.ndarray' object has no attribute 'tocsr'

In [28]:
import scipy.sparse as sp

In [29]:
sparse_user_items = sp.csr_matrix(user_items)

In [31]:
sparse_user_items.sum()

700722.0

In [32]:
import implicit

In [54]:
model = implicit.als.AlternatingLeastSquares(factors=50)

In [55]:
model.fit(sparse_user_items)

  0%|          | 0/15 [00:00<?, ?it/s]

In [64]:
recommendations = model.recommend(np.arange(len(clients_names)), sparse_user_items)

In [72]:
final_recommendations = []

for i, name in enumerate(clients_names):
    final_recommendations.append([name, goods_names[recommendations[0][i, 0]], recommendations[1][i, 0]])

In [73]:
final_recommendations = pd.DataFrame(final_recommendations, columns=['Клиент', 'Группа4', 'Score'])

In [76]:
final_recommendations.sort_values(['Score'], ascending=False).to_excel('next_best_offer.xlsx', index=None)